# Learning SHA-256 Cryptographic Hash Algorithm
## https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

In [32]:
import numpy as np

input_string = "hash this"

In [33]:
K = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
            0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
            0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
            0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
            0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
            0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

In [34]:
def sigma_1(x: int):
    out = (rotr(17, x) ^ rotr(19, x) ^ shr(10, x))
    return out

In [35]:
def sigma_0(x: int):
    out = (rotr(7, x) ^ rotr(18, x) ^ shr(3, x))
    return out

In [36]:
def sum_1(x: int):
    out = (rotr(6, x) ^ rotr(11, x) ^ rotr(25, x))
    return out

In [37]:
def sum_0(x: int):
    out = (rotr(2, x) ^ rotr(13, x) ^ rotr(22, x))
    return out

In [38]:
def rotr(n: int, x: int):
    w = 8 * 4
    out = ((x >> n) | (x << w - n))
    return out

In [39]:
def shr(n: int, x: int):
    out = x >> n
    return out

In [40]:
def ch(x: int, y: int, z: int):
    out = (x & y) ^ (~x & z)
    return out

In [41]:
def maj(x: int, y: int, z: int):
    out = (x & y) ^ (x & z) ^ (y & z)
    return out

In [42]:
# padding

# converting to binary
binary = bytearray(input_string, 'ascii')
l = len(binary) * 8
binary.append(0x80)

while (len(binary)*8 + 64)%512 != 0:
    binary.append(0x00)

binary += l.to_bytes(8, byteorder='big')
    
if (len(binary)*8) % 512 == 0:
    print("padding was successful")

padding was successful


In [43]:
# parsing
M = []

for i in range(0,len(binary), 64):
    M.append(binary[i:i+64])

In [44]:
# inital hash value
H = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a, 0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]

In [45]:
# hash computation
for message_block in M:
    W = []
    for t in range(0,64):
        if t <= 15:
            W.append(bytes(message_block[t*4:(t*4)+4]))
        else:
            W.append(sigma_1(int.from_bytes(W[t-2], 'big')) + int.from_bytes(W[t-7], 'big') + 
                     sigma_0(int.from_bytes(W[t-15], 'big')) + int.from_bytes(W[t-16], 'big'))
            W[t] = (W[t]% 2**32).to_bytes(4, 'big')
            
    a = H[0]
    b = H[1]
    c = H[2]
    d = H[3]
    e = H[4]
    f = H[5]
    g = H[6]
    h = H[7]
    
    for t in range (0, 64):
        t1 = ((h + sum_1(e) + ch(e, f, g) + K[t] + int.from_bytes(W[t], 'big')) % 2**32)
        t2 = ((sum_0(a) + maj(a, b, c)) % 2**32)
        h = g
        g = f 
        f = e
        e = ((d + t1) % 2**32)
        d = c
        c = b
        b = a
        a = ((t1 + t2) % 2**32)
  
    H[0] = (a + H[0]) % 2**32
    H[1] = (b + H[1]) % 2**32
    H[2] = (c + H[2]) % 2**32
    H[3] = (d + H[3]) % 2**32
    H[4] = (e + H[4]) % 2**32
    H[5] = (f + H[5]) % 2**32
    H[6] = (g + H[6]) % 2**32
    H[7] = (h + H[7]) % 2**32
    
    #for i in range(0, len(H)):
    #    H[i] = H[i] % 2**32

result = ((H[0]).to_bytes(4, 'big') + (H[1]).to_bytes(4, 'big') +
            (H[2]).to_bytes(4, 'big') + (H[3]).to_bytes(4, 'big') +
            (H[4]).to_bytes(4, 'big') + (H[5]).to_bytes(4, 'big') +
            (H[6]).to_bytes(4, 'big') + (H[7]).to_bytes(4, 'big')).hex()

print("'" + input_string + "'" + ' hashed using SHA 256 is: ' + result)

'hash this' hashed using SHA 256 is: 19467788bc0cf11790a075ea718452cecf0e79db59d1964670475e5fe2e4a611
